In [32]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from src.lib.FFNNClassifier import FFNNClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


In [33]:
def one_hot_encode(y, num_classes=10):
    """
    More robust one-hot encoding using NumPy

    Parameters:
    y (array-like): Input labels
    num_classes (int): Number of classes to encode

    Returns:
    numpy.ndarray: One-hot encoded array
    """
    # Ensure y is a NumPy array of integers
    y = np.asarray(y, dtype=int)

    # Create one-hot encoded array
    one_hot = np.zeros((len(y), num_classes))
    one_hot[np.arange(len(y)), y] = 1

    return one_hot

X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

# Type conversion
X = X.astype('float32')
y = y.astype('int')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# One-hot encode labels
y_train_one_hot = one_hot_encode(y_train)
y_test_one_hot = one_hot_encode(y_test)

In [34]:
def train_and_evaluate_mlp(hidden_layer_sizes, X_train, y_train, X_test, y_test):
    """
    Train MLP and return model, accuracy, and training loss
    """
    mlp = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        max_iter=50,  # Increased iterations for better convergence
        solver='adam',
        random_state=42,
        early_stopping=True,  # Use validation set for early stopping
        validation_fraction=0.1,  # 10% of training data for validation
        n_iter_no_change=5  # Stop if no improvement for 5 iterations
    )

    # Fit the model
    mlp.fit(X_train, y_train)

    # Predict and calculate accuracy
    y_pred = mlp.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    return mlp, accuracy, mlp.loss_curve_

In [35]:
def model_comparison(sk_mlp: MLPClassifier, custom_mlp: FFNNClassifier, is_weight: bool = False):

    if not is_weight:
        sk_mlp.fit(X_train_scaled, y_train)
    sk_pred = sk_mlp.predict(X_test_scaled)
    sk_accuracy = accuracy_score(y_test, sk_pred)
    print("[SKLEARN] Prediction: ",sk_pred)
    print("[SKLEARN] Accuracy: ", sk_accuracy)
    # print("[SKLEARN] Weights: ", sk_mlp.coefs_)
    # print("[SKLEARN] Bias: ", sk_mlp.intercepts_)
    print()

    if not is_weight:
        custom_mlp.fit(X_train_scaled, y_train_one_hot)
    custom_pred = custom_mlp.predict(X_test_scaled)
    y_test_labels = np.argmax(y_test_one_hot, axis=1)
    custom_accuracy = accuracy_score(y_test_labels, custom_pred)
    print("[CUSTOM] Prediction: ",custom_pred)
    print("[CUSTOM] Accuracy: ", custom_accuracy)
    # print("[SKLEARN] Weights: ", custom_mlp.weights_history)
    # print("[SKLEARN] Bias: ", custom_mlp.biases_history[-1])
    print()


In [36]:

"""
Pengaruh depth (banyak layer) dan width (banyak neuron per layer)
"""

# Configuration for depth variation (keeping width constant)
depth_configs = [
    (10, 10),  # 2 layers
    (10, 10, 10),  # 3 layers
    (10, 10, 10, 10)  # 4 layers
]

# Configuration for width variation (keeping depth constant)
width_configs = [
    (5,),  # narrow layer
    (15,),  # medium layer
    (30,)  # wide layer
]

results = {
    'depth_variations': [],
    'width_variations': []
}

# Experiment for depth variations (keeping width fixed)
print("Depth Variations Experiment:")
for depth_config in depth_configs:
    print(f"\nTesting depth configuration: {depth_config}")

    # Scikit-learn MLP
    sk_mlp = MLPClassifier(
        hidden_layer_sizes=depth_config,
        max_iter=20,
        random_state=42,
        learning_rate_init=0.01
    )

    # Custom MLP
    custom_mlp = FFNNClassifier(
        max_epoch=20,
        learning_rate=0.01,
        hidden_layer_sizes=depth_config,
        init_method="normal",
        mean=5.39294405e-05,
        std=.44,
        seed=69

    )

    model_comparison(sk_mlp, custom_mlp)

print("Width Variations Experiment:")
for width_configs in width_configs:
    print(f"\nTesting width configuration: {width_configs}")

    # Scikit-learn MLP
    sk_mlp = MLPClassifier(
        hidden_layer_sizes=width_configs,
        max_iter=20,
        random_state=42,
        learning_rate_init=0.01
    )

    # Custom MLP
    custom_mlp = FFNNClassifier(
        max_epoch=20,
        learning_rate=0.01,
        hidden_layer_sizes=width_configs,
        init_method="normal",
        mean=5.39294405e-05,
        std=.44,
        seed=69

    )

    model_comparison(sk_mlp, custom_mlp)


Depth Variations Experiment:

Testing depth configuration: (10, 10)


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[SKLEARN] Prediction:  [8 4 8 ... 2 7 1]
[SKLEARN] Accuracy:  0.9295

[CUSTOM] Prediction:  [5 6 5 ... 6 6 5]
[CUSTOM] Accuracy:  0.121


Testing depth configuration: (10, 10, 10)


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[SKLEARN] Prediction:  [8 4 6 ... 2 7 1]
[SKLEARN] Accuracy:  0.9293571428571429

[CUSTOM] Prediction:  [6 6 6 ... 6 6 6]
[CUSTOM] Accuracy:  0.09414285714285714


Testing depth configuration: (10, 10, 10, 10)


...

C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[SKLEARN] Prediction:  [8 4 5 ... 2 7 1]
[SKLEARN] Accuracy:  0.9401428571428572

[CUSTOM] Prediction:  [8 9 9 ... 9 3 6]
[CUSTOM] Accuracy:  0.0995


Testing width configuration: (30,)


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[SKLEARN] Prediction:  [8 4 8 ... 2 7 1]
[SKLEARN] Accuracy:  0.9547142857142857

[CUSTOM] Prediction:  [9 9 9 ... 9 9 9]
[CUSTOM] Accuracy:  0.10278571428571429



C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\lib\FFNNClassifier.py:84: RuntimeWarning: overflow encountered in exp
  elif func == 'sigmoid': return 1.0/(1.0 + np.exp(-x))


In [37]:
activation_configs = [
    ('linear','identity'),
    ('relu','relu'),
    ('sigmoid','logistic'),
    ('tanh','tanh')
]

print("Width Variations Experiment:")
for act_custom, act_sklearn in activation_configs:
    print(f"\nTesting activation configuration: {act_custom}")

    # Scikit-learn MLP
    sk_mlp = MLPClassifier(
        hidden_layer_sizes=[10],
        activation= act_sklearn,
        max_iter=20,
        random_state=42,
        learning_rate_init=0.01
    )

    # Custom MLP
    custom_mlp = FFNNClassifier(
        max_epoch=20,
        learning_rate=0.01,
        activation_func=[act_custom,act_custom],
        hidden_layer_sizes=[10],
        init_method="normal",
        mean=5.39294405e-05,
        std=.44,
        seed=69

    )
    model_comparison(sk_mlp, custom_mlp)

Width Variations Experiment:

Testing activation configuration: linear


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[SKLEARN] Prediction:  [8 4 5 ... 2 7 1]
[SKLEARN] Accuracy:  0.9067142857142857

[CUSTOM] Prediction:  [0 0 0 ... 0 0 0]
[CUSTOM] Accuracy:  0.09592857142857143


Testing activation configuration: relu


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[SKLEARN] Prediction:  [8 4 8 ... 2 7 1]
[SKLEARN] Accuracy:  0.9294285714285714

[CUSTOM] Prediction:  [0 0 0 ... 0 0 0]
[CUSTOM] Accuracy:  0.09585714285714286


Testing activation configuration: sigmoid


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[SKLEARN] Prediction:  [8 4 8 ... 2 7 1]
[SKLEARN] Accuracy:  0.9096428571428572

[CUSTOM] Prediction:  [2 5 5 ... 5 9 9]
[CUSTOM] Accuracy:  0.09235714285714286


Testing activation configuration: tanh


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\src\lib\FFNNClassifier.py:99: RuntimeWarning: overflow encountered in exp
  p = 2.0/(np.exp(x) - np.exp(-x)) # should be the same as 1 - np.tanh(x) ** 2. will check later


[SKLEARN] Prediction:  [8 4 8 ... 2 7 1]
[SKLEARN] Accuracy:  0.9026428571428572

[CUSTOM] Prediction:  [1 0 5 ... 0 2 0]
[CUSTOM] Accuracy:  0.036642857142857144



In [38]:
learning_rates = [0.1, 0.01, 0.001, 0.0001]

print("Width Variations Experiment:")
for rate in learning_rates:
    print(f"\nTesting activation configuration: {rate}")

    # Scikit-learn MLP
    sk_mlp = MLPClassifier(
        hidden_layer_sizes=[10],
        max_iter=20,
        random_state=42,
        learning_rate_init=rate
    )

    # Custom MLP
    custom_mlp = FFNNClassifier(
        max_epoch=20,
        learning_rate=rate,
        hidden_layer_sizes=[10],
        init_method="normal",
        mean=5.39294405e-05,
        std=.44,
        seed=69

    )

    model_comparison(sk_mlp, custom_mlp)

Width Variations Experiment:

Testing activation configuration: 0.1


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[SKLEARN] Prediction:  [2 4 2 ... 2 7 1]
[SKLEARN] Accuracy:  0.5694285714285714

[CUSTOM] Prediction:  [6 9 5 ... 5 9 9]
[CUSTOM] Accuracy:  0.09585714285714286


Testing activation configuration: 0.01


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[SKLEARN] Prediction:  [8 4 8 ... 2 7 1]
[SKLEARN] Accuracy:  0.9294285714285714

[CUSTOM] Prediction:  [2 5 5 ... 5 9 9]
[CUSTOM] Accuracy:  0.09235714285714286


Testing activation configuration: 0.001


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[SKLEARN] Prediction:  [8 4 8 ... 2 7 1]
[SKLEARN] Accuracy:  0.9322857142857143

[CUSTOM] Prediction:  [2 5 5 ... 5 9 9]
[CUSTOM] Accuracy:  0.091


Testing activation configuration: 0.0001


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[SKLEARN] Prediction:  [8 4 8 ... 2 7 1]
[SKLEARN] Accuracy:  0.9120714285714285

[CUSTOM] Prediction:  [5 5 5 ... 5 9 9]
[CUSTOM] Accuracy:  0.093



In [39]:
import random

weight_configs =  ['normal', 'zero', 'uniform']

print("Width Variations Experiment:")
for weight_config in weight_configs:
    print(f"\nTesting weight configuration: {weight_config}")

    lower_bound=5.39294405e-05
    upper_bound=1
    mean=5.39294405e-05
    std=.44
    seed=69

    # Scikit-learn MLP
    sk_mlp = MLPClassifier(
        hidden_layer_sizes=[10],
        max_iter=20,
        random_state=42,
        learning_rate_init=0.01
    )
    sk_mlp.fit(X_train_scaled, y_train)
    if weight_config == 'normal':
        for i in range(len(sk_mlp.coefs_)):
            sk_mlp.coefs_[i] = np.random.normal(mean, std, sk_mlp.coefs_[i].shape)
            sk_mlp.intercepts_[i] = np.random.normal(mean, std, sk_mlp.intercepts_[i].shape)
    elif weight_config == 'zero':
        for i in range(len(sk_mlp.coefs_)):
            sk_mlp.coefs_[i] = np.zeros(sk_mlp.coefs_[i].shape)
            sk_mlp.intercepts_[i] = np.zeros(sk_mlp.intercepts_[i].shape)
    elif weight_config == 'uniform':
        for i in range(len(sk_mlp.coefs_)):
            sk_mlp.coefs_[i] = np.random.uniform(lower_bound, upper_bound, sk_mlp.coefs_[i].shape)
            sk_mlp.intercepts_[i] = np.random.uniform(lower_bound, upper_bound, sk_mlp.intercepts_[i].shape)
    else:
        raise ValueError('Unknown weight configuration')



    # Custom MLP
    custom_mlp = FFNNClassifier(
        max_epoch=20,
        learning_rate=0.01,
        hidden_layer_sizes=[10],
        init_method="normal",
        lower_bound=lower_bound,
        upper_bound=upper_bound,
        mean=mean,
        std=std,
        seed=seed
    )
    custom_mlp.fit(X_train_scaled, y_train_one_hot)


    model_comparison(sk_mlp, custom_mlp, True)

Width Variations Experiment:

Testing weight configuration: normal


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[SKLEARN] Prediction:  [3 5 4 ... 3 9 3]
[SKLEARN] Accuracy:  0.08785714285714286

[CUSTOM] Prediction:  [2 5 5 ... 5 9 9]
[CUSTOM] Accuracy:  0.09235714285714286


Testing weight configuration: zero


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[SKLEARN] Prediction:  [0 0 0 ... 0 0 0]
[SKLEARN] Accuracy:  0.09592857142857143

[CUSTOM] Prediction:  [2 5 5 ... 5 9 9]
[CUSTOM] Accuracy:  0.09235714285714286


Testing weight configuration: uniform


C:\Users\ASUS\Documents\GitHub\Feedforward-Neural-Network\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[SKLEARN] Prediction:  [8 4 1 ... 0 0 0]
[SKLEARN] Accuracy:  0.0735

[CUSTOM] Prediction:  [2 5 5 ... 5 9 9]
[CUSTOM] Accuracy:  0.09235714285714286

